In [ ]:
using Plots
using FFTW

@userplot Evolve
@recipe function f(ev::Evolve)
    x, φt, i = ev.args
    yrange --> (-1.5, 1.5)
    x, φt[i,:]
end

In [ ]:
L = 100
N = 1000
dx = L/N
dk = 2*π / L

x = dx .* collect(0:N-1)
k = dk .* collect(0:N÷2)

dt = 1e-1

k² = k.^2
c = -k.^4
cdt = c .* dt
k1 = exp.(cdt)
k2 = dt * ( exp.(cdt) .- 1 ) ./ cdt
k3 = dt * ( exp.(cdt) .- (1 .+ cdt) ) ./ cdt.^2
k2[1] = 1
k3[1] = 0

F = plan_rfft(x)
B = plan_irfft(k.*im, N)

g(φ) = k² .* (F * (φ .- φ.^3))

In [ ]:
function antialias!(F)
    F[end-(N÷6)+1:end] .= 0.
end

function etd!(Fφ, φ)
    g0 = g(φ)
    a = @. k1 * Fφ + k2 * g0
    φ .= B*a
    g1 = g(φ)
    @. Fφ = a + k3 * (g1 - g0)
    antialias!(Fφ)
    φ .= B*Fφ
end

In [ ]:
v = .3
φ = v * ones(N) + cos.((2π/L).*x) * 1e-1
Fφ = F*φ
M = Int(1e6)
frames = 500
n = M ÷ frames

φt = Array{Float64}(undef, (frames, N))
φt[1,:] .= φ

for i in 2:frames
    for j in 1:n
        etd!(Fφ, φ)
    end
    φt[i,:] .= φ
end

In [ ]:
anim = @animate for i in 1:frames
    evolve(x, φt, i)
end
gif(anim, "anim_fps2.mp4", fps = 30)